## 18BCE2522

In [7]:
#PACKAGES
import nltk
import re
import numpy as np
from gensim import corpora, similarities
from gensim.models import LsiModel
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shash\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [5]:
import nltk

## FUNCTIONS

In [8]:
#function to filter out stopwords and apply word stemming
def filter_words_and_get_word_stems(document, word_tokenizer, word_stemmer,
                                    stopword_set, pattern_to_match_words=r"[^\w]",
                                    word_length_minimum_n_chars=2):
    """Remove multiple white spaces and all non word content from text and
    extract words. Then filter out stopwords and words with a length smaller
    than word_length_minimum and apply word stemmer to get wordstems. Finally
    return word stems.
    """
    document = re.sub(pattern_to_match_words, r" ", document)
    document = re.sub(r"\s+", r" ", document)
    words = word_tokenizer.tokenize(document)
    words_filtered = [word.lower()
                      for word in words
                      if word.lower() not in stopword_set and len(word) >= word_length_minimum_n_chars]
    word_stems = [word_stemmer.lemmatize(word) for word in words_filtered]
    return(word_stems)

## INPUT

In [9]:
#training text data to calculate TF-IDF model from
documents_train = [
  "She goes to school.",
  "She runs to the shop.",
  "I go to school and he goes to the shop."]

#test data text data to match 
document_test = "He runs to school."

## PREPROCESS

In [21]:
#set stopword set, word stemmer and word tokenizer
stopword_set = set(stopwords.words("english"))
word_tokenizer = nltk.tokenize.WordPunctTokenizer()
word_stemmer =  nltk.WordNetLemmatizer()

#apply cleaning, filtering and word stemming to training documents
word_stem_arrays_train = [
        filter_words_and_get_word_stems(
                str(document),
                word_tokenizer,
                word_stemmer,
                stopword_set
                ) for document in documents_train]
print("Word Stems of Training Documents:", word_stem_arrays_train)
print("")

#apply cleaning, filtering and word stemming to test document
word_stem_array_test = filter_words_and_get_word_stems(
        document_test,
        word_tokenizer,
        word_stemmer,
        stopword_set)
print("Word Stems of Test Document:", word_stem_array_test)

Word Stems of Training Documents: [['go', 'school'], ['run', 'shop'], ['go', 'school', 'go', 'shop']]

Word Stems of Test Document: ['run', 'school']


## PROCESS

In [23]:
#create dictionary containing unique word stems of training documents
#TF (term frequencies) or "global weights"
dictionary = corpora.Dictionary(
  word_stem_array_train
  for word_stem_array_train in word_stem_arrays_train)
print("Dictionary :", dictionary)
print("")

Dictionary : Dictionary(4 unique tokens: ['go', 'school', 'run', 'shop'])



In [24]:
#create corpus containing word stem id from dictionary and word stem count
#for each word in each document
#DF (document frequencies, for all terms in each document) or "local weights"
corpus = [
  dictionary.doc2bow(word_stem_array_train)
  for word_stem_array_train in word_stem_arrays_train]
print("Corpus :", corpus)
print("")

Corpus : [[(0, 1), (1, 1)], [(2, 1), (3, 1)], [(0, 2), (1, 1), (3, 1)]]



In [25]:
#create LSI model (Latent Semantic Indexing) from corpus and dictionary
#LSI model consists of Singular Value Decomposition (SVD) of
#Term Document Matrix M: M = T x S x D'
#and dimensionality reductions of T, S and D ("Derivation")
lsi_model = LsiModel(
        corpus=corpus,
        id2word=dictionary #, num_topics = 2 #(opt. setting for explicit dim. change)
        )
print("Derivation of Term Matrix T of Training Document Word Stems: \n",
      lsi_model.get_topics())
print("")

Derivation of Term Matrix T of Training Document Word Stems: 
 [[ 0.79411857  0.47930412  0.05482989  0.36964434]
 [-0.2236068  -0.2236068   0.67082039  0.67082039]
 [-0.26339267  0.68576057  0.54497127 -0.40418197]]



In [26]:
#Derivation of Term Document Matrix of Training Document Word Stems = M' x [Derivation of T]
print("LSI Vectors of Training Document Word Stems: \n",
      [lsi_model[document_word_stems] for document_word_stems in corpus])
print("")

LSI Vectors of Training Document Word Stems: 
 [[(0, 1.2734226922603291), (1, -0.447213595499958), (2, 0.42236790460030676)], [(0, 0.4244742307534435), (1, 1.3416407864998736), (2, 0.14078930153343583)], [(0, 2.4371856025006933), (2, -0.24520672699445256)]]



In [28]:
#calculate cosine similarity matrix for all training document LSI vectors
cosine_similarity_matrix = similarities.MatrixSimilarity(lsi_model[corpus])
print("Cosine Similarities of LSI Vectors of Training Documents: \n",
      [row for row in cosine_similarity_matrix])
print("")

Cosine Similarities of LSI Vectors of Training Documents: 
 [array([ 1.0000000e+00, -1.9446199e-08,  8.6602545e-01], dtype=float32), array([-1.9446199e-08,  1.0000000e+00,  2.8867516e-01], dtype=float32), array([0.86602545, 0.28867516, 1.        ], dtype=float32)]



In [29]:
#calculate LSI vector from word stem counts of the test document and the LSI model content
vector_lsi_test = lsi_model[dictionary.doc2bow(word_stem_array_test)]
print("LSI Vector Test Document:", vector_lsi_test)
print("")

LSI Vector Test Document: [(0, 0.5341340127734087), (1, 0.44721359549995776), (2, 1.230731837728502)]



In [31]:
#perform a similarity query against the corpus
cosine_similarities_test = cosine_similarity_matrix[vector_lsi_test]
print("Cosine Similarities of Test Document LSI Vectors to Training Documents LSI Vectors:\n",
      cosine_similarities_test)
print("")

Cosine Similarities of Test Document LSI Vectors to Training Documents LSI Vectors:
 [0.49999997 0.5        0.28867513]



## OUTPUT

In [32]:
#get text of test documents most similar training document
most_similar_document_test = documents_train[np.argmax(cosine_similarities_test)]
print("Most similar Training Document to Test Document:\n", most_similar_document_test)

Most similar Training Document to Test Document:
 She runs to the shop.
